## Data Analysis and Cleaning for Book Bans from the PEN dataset 
found here: https://docs.google.com/spreadsheets/d/1a6v7R7pidO7TIwRZTIh9T6c0--QNNVufcUUrDcz2GJM/edit#gid=717219603


In [ ]:
# download dataset 
import pandas as pd

dataframe_bans = pd.read_csv('ban_data/Pen_ban_data.csv')

dataframe_bans

## Reuse the same code from book_bans_summary_table to create summary table 

I need two things from this dataset 
1. response 
2. year either 2022 or 2023, probs just have one for both and one for total 

The data follows this pattern: 

**Challenge Information**

1. Date of Challenge/Removal:	The approximate month and year when a book was first noted as removed or prohibited based on available public documents.

2. Type of Ban	Indicates one of four types of bans: 'Banned from Libraries and Classrooms,' 'Banned from Libraries,' 'Banned from Classrooms,' or 'Banned Pending Investigation.'

* Banned from Libraries and Classrooms: These represent instances where individual titles were placed off-limits for students in either some or all libraries and classrooms.

* Banned from Libraries	These represent instances in which administrators or school boards have removed individual titles from school libraries where they were previously available. Books in this category are not necessarily banned from classroom curriculum. This category includes decisions to ban a book from one school-level library (e.g. a middle school) even if it is included in libraries for higher grades (e.g. a high school), or other forms of grade-level restrictions.
* Banned from Classrooms	These represent instances where school boards or other school authorities have barred individual titles from classroom libraries, curriculum, or optional reading lists. These constitute bans on use in classrooms, even in cases where the books may still be available in school libraries.
* Banned Pending Investigation	These are instances where a title was removed from access and no final decision has been made on the availability of the material. The Index records these bans, even if only temporarily enforced and even if books have ultimately been returned to shelves.

3. Origin of Challenge The approximate initial action that led to the removal/reduction of access to a title. 
	

In [ ]:
import pandas as pd

# uses the definitions about what states have done if the definition is unknown 
# for the response the book could still be challenged and just nothing has happened to it 



def create_summary(response, df, col:str, geo_res):
    """ geo_res: geographic resolution either 'State' tag or 'District' tag
        to group summary by fields for a particular place, if district is chosen
        need to ensure state tag also still persists 
    """
    # Filter the DataFrame based on the response
    filtered_df = df[df[col] == response]
    
    # Group the filtered DataFrame by the 'State' column and count the number of entries in each group
    summary = filtered_df.groupby(geo_res).size().reset_index(name='Count_of_{}_Books'.format(response))
    
    # If some resolutions have no books for this response, fill those with 0 TODO: change state resolution to full 
    all_states = df[geo_res].unique()
    missing_states = set(all_states) - set(summary[geo_res].unique())
    missing_summary = pd.DataFrame({'State': list(missing_states), 'Count_of_{}_Books'.format(response): [0] * len(missing_states)})
    
    # Concatenate summary and missing_summary to ensure all states are included
    summary = pd.concat([summary, missing_summary])
    
    # Reset index
    summary.reset_index(drop=True, inplace=True)
    
    return summary


def create_summary_table(column: str, response_types:list, geo_res:str) -> pd.DataFrame:
        
    # Create a list of DataFrames containing summaries for each response type
    summary_list = [create_summary(response, dataframe_bantable, column, geo_res) for response in response_types]
    
    # Merge DataFrames horizontally based on the 'State' column
    table = pd.concat(summary_list, axis=1)
    
    # Drop duplicate 'State' columns (if any)
    table = table.loc[:, ~table.columns.duplicated()]
    
    # add a sum of all the types of bans to the end of the row 
    table["sum_total_challenges_bans"] = table.iloc[:, 1:].sum(axis=1)
    
    # Find the most occurring book for each state
    most_occuring_books = dataframe_bantable.groupby(geo_res)['Title'].agg(
        lambda x: (x.value_counts().iloc[0], x.value_counts().index[0]) if not x.empty else (0, None)
    ).reset_index(name='Most_Occurring_Book')
    
    # Merge the most occurring book information with the summary
    table = pd.merge(table, most_occuring_books, on=geo_res, how='left')
    
    return table



In [ ]:

#TODO: Include origin of challenge as seperate categories as well? 

response_types = ['Banned from Libraries and Classrooms', 
                  'Banned from Libraries'
                  ,'Banned from Classrooms','Banned Pending Investigation']


In [ ]:
# resolution for a single entry can be 
# AY all years 
# or put in Date format MO YR 


In [ ]:
# for all states for years 2022 - 2023

# rock this for individual states 
states_decision_summary22_23 = create_summary_table('Ban Status', response_types, geo_res='State')

#counties_decision_sumary
counties_decision_summary22_23 = create_summary_table('Ban Status', response_types, geo_res='District')